In [2]:
import pandas as pd
import numpy as np
import time 

In [40]:
parquet_file = pd.read_parquet('ciciot2022.parquet')
parquet_file

,L4_tcp,L4_udp,L7_http,L7_https,port_class_src,port_class_dst,pck_size,ip_dst_new,ethernet_frame_size,ttl,...,q3_e,q1_e,iqr_e,epoch_timestamp,inter_arrival_time,time_since_previously_displayed_frame,global_category,device,interaction_type,command
0,True,False,False,False,2,3,442,3232270593,476,64,...,442.00,442.00,0.00,1634049773,0.000000,0.000000,AUDIO,nestmini,LAN,VOLUMEON
1,True,False,False,False,3,2,20,3232270754,54,128,...,336.50,125.50,211.00,1634049773,0.048638,0.048638,AUDIO,nestmini,LAN,VOLUMEON
2,False,False,False,False,0,0,64,3232270593,98,64,...,253.00,42.00,211.00,1634049773,0.386110,0.434748,AUDIO,nestmini,LAN,VOLUMEON
3,True,False,False,False,2,3,300,3232270593,334,64,...,335.50,53.00,282.50,1634049774,0.458671,0.893419,AUDIO,nestmini,LAN,VOLUMEON
4,True,False,False,False,3,2,20,3232270754,54,128,...,300.00,20.00,280.00,1634049774,0.051143,0.944562,AUDIO,nestmini,LAN,VOLUMEON
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229501,True,False,False,False,2,2,89,3232270836,123,223,...,157.25,89.00,68.25,1633459940,0.088585,0.088585,HOME AUTOMATION,heimvisionlamp,LOCAL,LIGHTOFF
229502,True,False,False,False,2,2,20,597822746,54,255,...,134.50,54.50,80.00,1633459940,0.076676,0.165261,HOME AUTOMATION,heimvisionlamp,LOCAL,LIGHTOFF
229503,False,True,False,False,3,2,180,4294967295,214,255,...,180.00,71.75,108.25,1633459940,0.545729,0.710990,HOME AUTOMATION,heimvisionlamp,LOCAL,LIGHTOFF
229504,False,True,False,False,3,2,180,4294967295,214,255,...,180.00,89.00,91.00,1633459945,5.005840,5.716830,HOME AUTOMATION,heimvisionlamp,LOCAL,LIGHTOFF


In [3]:
parquet_file.to_csv('ciciot2022.csv')

In [3]:
csv_file = pd.read_csv('ciciot2022.csv')

In [5]:
#Esta columna se agrega al pasar el dataset a formato CSV
csv_file = csv_file.drop(columns= ['Unnamed: 0']) 

In [6]:
csv_file = csv_file.drop(columns = ['ip_dst_new','source_port', 'dest_port', 'most_freq_d_ip',
                                   'most_freq_sport', 'most_freq_dport', 'epoch_timestamp'])

In [7]:
#Vemos que hay demasiadas filas de tipo "camera"
csv_file.global_category.value_counts()

CAMERA             191175
HOME AUTOMATION     19557
AUDIO               18774
Name: global_category, dtype: int64

In [26]:
df_bal = csv_file.drop(csv_file.index[36107:209949], inplace=False) #Balanceamos

In [27]:
#Vemos que ahora está algo más balanceado. 
df_bal.global_category.value_counts()
#NO CORTAMOS NINGÚN FLUJO
#df_bal.loc[df_bal.global_category == 'CAMERA']

HOME AUTOMATION    19557
AUDIO              18774
CAMERA             17333
Name: global_category, dtype: int64

In [28]:
df_bal.to_parquet('dataset_bal.parquet', use_dictionary = False)

In [29]:
df_bal.to_csv('dataset_bal.csv')

In [30]:
#Cargamos el dataset balanceado
df = pd.read_parquet('dataset_bal.parquet')

# Generamos un nuevo dataset únicamente con los dispositivos de audio.

In [43]:
audio_df = df.loc[df.global_category == 'AUDIO']


In [44]:
#Balanceamos AUDIO (17519)
audio_df = audio_df.drop(audio_df.index[17520:18774], inplace=False)
#audio_df.to_csv('Audio.csv')

In [46]:
#Creamos el conjunto de Train de los dispositivos de Audio (81%)
audio_train = audio_df.drop(audio_df.index[14258:17520], inplace=False)

In [48]:
#Creamos el conjunto de Test de los dispositivos de Audio (19%)
audio_test = audio_df.drop(audio_df.index[0:14258], inplace=False)

In [49]:
print(f'Audio Train : {audio_train.shape}')
print(f'Audio Test : {audio_test.shape}')

Audio Train : (14258, 45)
Audio Test : (3262, 45)


In [44]:
audio_train.to_parquet('Audio_train.parquet', use_dictionary = False)
audio_test.to_parquet('Audio_test.parquet', use_dictionary = False)

In [47]:
audio_train_df = pd.read_parquet('Audio_train.parquet')

# Generaremos un dataset únicamente con los dispositivos que sean cámaras.

In [50]:
camera_df = df.loc[df.global_category == 'CAMERA']
camera_df.to_csv('Camera.csv')

In [51]:
print(f'Camera : {camera_df.shape}')

Camera : (17333, 45)


In [52]:
camera_train = camera_df.drop(camera_df.index[13932:17333], inplace=False) #80%
camera_test = camera_df.drop(camera_df.index[0:13932], inplace=False) #20%

In [76]:
camera_train.to_parquet('Camera_train.parquet', use_dictionary = False)
camera_test.to_parquet('Camera_test.parquet', use_dictionary = False)

In [55]:
print(f'Camera Train : {camera_train.shape}')
print(f'Camera Test : {camera_test.shape}')

Camera Train : (13932, 45)
Camera Test : (3401, 45)


# Generaremos un dataset únicamente con los dispositivos de automatización.


In [56]:
HA_df = df.loc[df.global_category == 'HOME AUTOMATION']

In [57]:
#Balanceamos HA (17331)
HA_df = HA_df.drop(HA_df.index[17331:19559], inplace=False)
HA_df.to_csv

In [58]:
HA_train = HA_df.drop(HA_df.index[14275:17331], inplace=False) #82%
HA_test = HA_df.drop(HA_df.index[0:14275], inplace=False) #18%

In [85]:
HA_train.to_parquet('HA_train.parquet', use_dictionary = False)
HA_test.to_parquet('HA_test.parquet', use_dictionary = False)

In [61]:
print(f'HA Train : {HA_train.shape}')
print(f'HA Test : {HA_test.shape}')

HA Train : (14275, 45)
HA Test : (3056, 45)


# CONCATENAMOS LOS ARCHIVOS DE TRAIN Y TEST

In [62]:
train_df = pd.concat(objs=[audio_train, camera_train, HA_train], ignore_index=True, copy=False, sort=False)
test_df = pd.concat(objs=[audio_test, camera_test, HA_test], ignore_index=True, copy=False, sort=False)

,L4_tcp,L4_udp,L7_http,L7_https,port_class_src,port_class_dst,pck_size,ethernet_frame_size,ttl,total_length,...,var_e,q3_e,q1_e,iqr_e,inter_arrival_time,time_since_previously_displayed_frame,global_category,device,interaction_type,command
0,True,False,False,True,1,2,1480,1514,38,1500,...,474226.940,1460.00,32.0,1428.00,0.000000,0.000000,AUDIO,sonosone,ALEXA,PLAY
1,True,False,False,True,1,2,354,388,38,374,...,460358.120,1460.00,32.0,1428.00,0.000063,0.000063,AUDIO,sonosone,ALEXA,PLAY
2,True,False,False,True,2,1,32,66,64,52,...,466663.600,1460.00,32.0,1428.00,0.002075,0.002138,AUDIO,sonosone,ALEXA,PLAY
3,True,False,False,True,2,1,32,66,64,52,...,469437.720,1460.00,32.0,1428.00,0.000000,0.002138,AUDIO,sonosone,ALEXA,PLAY
4,True,False,False,True,2,1,71,105,64,91,...,0.000,71.00,71.0,0.00,0.016452,0.018590,AUDIO,sonosone,ALEXA,PLAY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9714,True,False,False,True,3,1,345,379,64,365,...,43057.184,318.25,32.0,286.25,0.160403,30.932533,HOME AUTOMATION,roomba,LAN,CLEAN
9715,True,False,False,True,1,3,757,791,228,777,...,56079.633,343.00,32.0,311.00,0.046492,30.979025,HOME AUTOMATION,roomba,LAN,CLEAN
9716,True,False,False,True,3,1,32,66,64,52,...,55646.800,334.75,32.0,302.75,0.001574,30.980598,HOME AUTOMATION,roomba,LAN,CLEAN
9717,False,False,False,False,0,0,0,60,0,0,...,0.000,0.00,0.0,0.00,3.703685,34.684284,HOME AUTOMATION,roomba,LAN,CLEAN


In [64]:
train_df

,L4_tcp,L4_udp,L7_http,L7_https,port_class_src,port_class_dst,pck_size,ethernet_frame_size,ttl,total_length,...,var_e,q3_e,q1_e,iqr_e,inter_arrival_time,time_since_previously_displayed_frame,global_category,device,interaction_type,command
0,True,False,False,False,2,3,442,476,64,462,...,0.00000,442.0,442.0,0.0,0.000000,0.000000,AUDIO,nestmini,LAN,VOLUMEON
1,True,False,False,False,3,2,20,54,128,40,...,44521.00000,336.5,125.5,211.0,0.048638,0.048638,AUDIO,nestmini,LAN,VOLUMEON
2,False,False,False,False,0,0,64,98,64,84,...,35878.22300,253.0,42.0,211.0,0.386110,0.434748,AUDIO,nestmini,LAN,VOLUMEON
3,True,False,False,False,2,3,300,334,64,320,...,29822.75000,335.5,53.0,282.5,0.458671,0.893419,AUDIO,nestmini,LAN,VOLUMEON
4,True,False,False,False,3,2,20,54,128,40,...,29423.36000,300.0,20.0,280.0,0.051143,0.944562,AUDIO,nestmini,LAN,VOLUMEON
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42460,False,True,False,False,2,2,56,90,64,76,...,100.00000,56.0,36.0,20.0,0.000000,41.149025,HOME AUTOMATION,eufyhomebase,WAN,RECORDING
42461,False,True,False,False,2,2,56,90,64,76,...,96.00000,56.0,36.0,20.0,0.000000,41.149025,HOME AUTOMATION,eufyhomebase,WAN,RECORDING
42462,False,True,False,False,2,2,20,54,38,40,...,188.88889,56.0,36.0,20.0,0.028515,41.177540,HOME AUTOMATION,eufyhomebase,WAN,RECORDING
42463,False,True,False,False,2,2,20,54,41,40,...,228.57143,56.0,28.0,28.0,0.017245,41.194786,HOME AUTOMATION,eufyhomebase,WAN,RECORDING


In [65]:
print(f'Train : {train_df.shape}') #81.37%
print(f'Test : {test_df.shape}') #18.6%

Train : (42465, 45)
Test : (9719, 45)


In [67]:
train_df.to_parquet('train.parquet', use_dictionary = False)
test_df.to_parquet('test.parquet', use_dictionary = False)